In [ ]:
# User defined working directory
work_dir = '/Users/Tem/Documents/naturalistic-threat-ptsd'

In [ ]:
# Import packages
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd
from datetime import datetime
from nilearn import image
from nilearn import input_data
from datetime import datetime
from nltools.data import Brain_Data, Design_Matrix
from nltools.stats import find_spikes 
from nltools.stats import zscore
import nibabel.imagestats

In [ ]:
# Load list of subIDs 
cohort_IDs = pd.read_csv(f"{work_dir}/data/subs/cohort_IDs.csv") 
sub_list = cohort_IDs['subject'].tolist()
nCohort = len(sub_list)

## Smooth fMRI data

In [ ]:
# User defined variables
unsmoothed_dir = f"{work_dir}/data/neural-naturalistic/niftis/naturalistic-prepped-unsmoothed"
smoothed_output_dir = f"{work_dir}/data/neural-naturalistic/niftis/naturalistic-prepped-smoothed-6mm" 
fwhm = 6

In [ ]:
# Smooth fMRI files
if os.path.exists(smoothed_output_dir) == False:
    os.makedirs(smoothed_output_dir)
    print(f"Created smoothed_output_dir: \n{smoothed_output_dir}")

unsmoothed_list = os.listdir(f"{unsmoothed_dir}")
unsmoothed_list = [x for x in unsmoothed_list if 'nii' in x]

for i in unsmoothed_list:
    subID = i.split('_')[0]
    startTime = datetime.now()
    unsmoothed_image = image.load_img(f"{unsmoothed_dir}/{i}")
    smoothed_image = image.smooth_img(unsmoothed_image, fwhm = fwhm)
    smoothed_image.to_filename(f"{smoothed_output_dir}/{i}.gz")
    print(f"{subID} smoothing runtime = {datetime.now() - startTime}")

## Generate design matrices

In [ ]:
# User defined variables
smoothed_dir = f"{work_dir}/data/neural-naturalistic/niftis/naturalistic-prepped-smoothed-6mm" 
confound_dir = f"{work_dir}/data/neural-naturalistic/confounds/raw-confounds/"
dm_output_dir = f"{work_dir}/data/neural-naturalistic/confounds/dm-6mm" 
outlier_cutoff = 3 #define outlier cutoff for despiking
TR = 1

In [ ]:
# Create lists of nifti and confound files
exp_smoothed_dir = os.listdir(f"{smoothed_dir}")
exp_confound_dir = os.listdir(f"{confound_dir}")

smoothed_list = [None] * nCohort
confound_list = [None] * nCohort

for i in range(nCohort):
    smoothed_list[i] = [x for x in exp_smoothed_dir if sub_list[i] in x][0]
    confound_list[i] = [x for x in exp_confound_dir if sub_list[i] in x][0]

In [ ]:
# Assemble full design matrices (with motion parameters) and save as tsv files 
if os.path.exists(dm_output_dir) == False:
    os.makedirs(dm_output_dir)
    print(f"Created dm_output_dir: \n{dm_output_dir}\n")
    
full_dms = [None] * nCohort

for i in range(nCohort):
    
    print(sub_list[i])
    
    # Create BrainData object
    print('creating braindata object...')
    startTime = datetime.now()
    smoothed_img = image.load_img(smoothed_dir + '/' + smoothed_list[i])
    bd_func_img = Brain_Data(smoothed_img)
    print(f"BrainData runtime = {datetime.now() - startTime}")
    
    # Identify spikes in fmri data
    spikes = bd_func_img.find_spikes(global_spike_cutoff = outlier_cutoff, diff_spike_cutoff = outlier_cutoff)
    spikes = spikes.drop(labels = 'TR', axis = 1)
    
    # Create design matrix
    covariates = pd.read_csv(confound_dir + '/' + confound_list[i], delimiter = "\t")
    covariates = covariates[['global_signal', 'trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']]
    covariates['intercept'] = 1
    dm = Design_Matrix(pd.concat([covariates, spikes], axis = 1), sampling_freq = 1/TR)
    
    # Save design matrix 
    output_dm_filename = f"{dm_output_dir}/{sub_list[i]}_design-mat.tsv"
    dm.to_csv(output_dm_filename, sep = '\t', index = False)
    print(f"{sub_list[i]} design matrix runtime = {datetime.now() - startTime}\n")

## Mask data by ROIs

In [ ]:
# User-defined variables
mask_dir = f"{work_dir}/masks/amygdala-harvard-oxford"
dm_dir = f"{work_dir}/data/neural-naturalistic/confounds/dm-6mm"
smoothed_dir = f"{work_dir}/data/neural-naturalistic/niftis/naturalistic-prepped-smoothed-6mm"
npy_output_dir = f"{work_dir}/data/neural-naturalistic/niftis/masked/amygdala-harvard-oxford-6mm"
save_suffix = '_6mm_data'
nTRs = 1020

In [ ]:
# Create directory to house output npy files
if os.path.exists(npy_output_dir) == False:
    os.makedirs(npy_output_dir)
    print(f"created {npy_output_dir}")

In [ ]:
# Create a list of the ROIs to be masked 
ROI_mask_list = os.listdir(f"{mask_dir}")
ROI_mask_list = [x for x in ROI_mask_list if 'nii' in x]
nROIs = len(ROI_mask_list)

print(ROI_mask_list)

In [ ]:
# Generate masker objects
all_maskers = [None] * len(ROI_mask_list)
all_ROI_data = [None] * len(ROI_mask_list)

first_sub = sub_list[0]
func_file = f"{smoothed_dir}/{first_sub}_prepped.nii.gz"
func_img = image.load_img(func_file)

for i in range(len(ROI_mask_list)):
    mask_img = image.load_img(f"{mask_dir}/{ROI_mask_list[i]}")
    nVoxels = nibabel.imagestats.count_nonzero_voxels(mask_img)
    if mask_img.shape != func_img.shape[0:3]:
        mask_img = image.resample_to_img(mask_img, func_img, interpolation = 'nearest')
    all_maskers[i] = input_data.NiftiMasker(mask_img = mask_img, 
                                    mask_strategy = 'epi', 
                                    standardize = True,
                                    detrend = True,
                                    low_pass = 0.1,
                                    high_pass = 0.01,
                                    t_r = 1)
    all_ROI_data[i] = np.zeros([nTRs, nVoxels, nCohort])
    
print(f"Generated {i + 1} maskers.")

In [ ]:
# Mask fMRI data for each subject using each mask
for i in range(nCohort):
    print(f"Masking {sub_list[i]} data...")
    func_file = f"{smoothed_dir}/{sub_list[i]}_prepped.nii.gz"
    func_img = image.load_img(func_file)
    confound_file = f"{dm_dir}/{sub_list[i]}_design-mat.tsv"
    confound_df = pd.read_csv(confound_file, delimiter = "\t")[:nTRs]
    
    for j in range(len(ROI_mask_list)):
        all_ROI_data[j][:, :, i] = all_maskers[j].fit_transform(func_img, confounds = confound_df)

In [ ]:
# Save masked data as npy arrays
for i in range(len(ROI_mask_list)):
    cROI = ROI_mask_list[i][:-9]
    cData = all_ROI_data[i]
    cData = np.transpose(cData, (1, 0, 2))
    np.save(f"{npy_output_dir}/{cROI}{save_suffix}", cData)